In [31]:
# https://xilinx.github.io/brevitas/getting_started.html

from torch.nn import Module
import torch.nn.functional as F

import brevitas.nn as qnn
from brevitas.quant import Int32Bias

import os
import onnx
import torch
import numpy as np
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader, Dataset
from brevitas.nn import QuantLinear, QuantReLU
import torch.nn as nn
from sklearn.metrics import accuracy_score
from tqdm import tqdm, trange

weight_bit_width = 2
act_bit_width = 2

class QuantWeightActBiasLeNet(Module):
    def __init__(self):
        super(QuantWeightActBiasLeNet, self).__init__()
        self.quant_inp = qnn.QuantIdentity(bit_width=4, return_quant_tensor=True)
        self.conv1 = qnn.QuantConv2d(3, 6, 5, bias=True, weight_bit_width=weight_bit_width, bias_quant=Int32Bias)
        self.relu1 = qnn.QuantReLU(bit_width=act_bit_width, return_quant_tensor=True)
        self.conv2 = qnn.QuantConv2d(6, 16, 5, bias=True, weight_bit_width=weight_bit_width, bias_quant=Int32Bias)
        self.relu2 = qnn.QuantReLU(bit_width=act_bit_width, return_quant_tensor=True)
        self.fc1   = qnn.QuantLinear(16*10*10, 120, bias=True, weight_bit_width=weight_bit_width, bias_quant=Int32Bias)
        # self.fc1   = qnn.QuantLinear(16*5*5, 120, bias=True, weight_bit_width=weight_bit_width, bias_quant=Int32Bias)
        self.relu3 = qnn.QuantReLU(bit_width=act_bit_width, return_quant_tensor=True)
        self.fc2   = qnn.QuantLinear(120, 84, bias=True, weight_bit_width=weight_bit_width, bias_quant=Int32Bias)
        self.relu4 = qnn.QuantReLU(bit_width=act_bit_width, return_quant_tensor=True)
        self.fc3   = qnn.QuantLinear(84, 10, bias=True, weight_bit_width=weight_bit_width, bias_quant=Int32Bias)

    def forward(self, x):
        out = self.quant_inp(x)
        out = self.relu1(self.conv1(out))
        out = F.max_pool2d(out, 2)
        out = self.relu2(self.conv2(out))
        # out = F.max_pool2d(out, 2)
        out = out.reshape(out.shape[0], -1)
        out = self.relu3(self.fc1(out))
        out = self.relu4(self.fc2(out))
        out = self.fc3(out)
        return out

model = QuantWeightActBiasLeNet()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Target device: " + str(device))

model.to(device); # The semicolon is for not printing the model.

Target device: cpu


In [4]:
# This is for 4-bit quantization. The quantization value can be changed in line 11. 

import numpy as np
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
import os

quant_param = 4 # log2(quant_param) bits. #256 means no quanization, 1 means 1-bit quantization

def quantize_image(image):
    """Quantize and binarize an image."""
    image = image.astype(np.float32)
    image = np.floor(image / (256/quant_param))  # Example: reducing to 4-bit quantization
    return image.astype(np.float32)

def save_dataset_as_npz(data, labels, filename):
    """Save the dataset as a .npz file."""
    np.savez_compressed(filename, data=data, labels=labels)

# Load CIFAR-10 dataset
transform = transforms.Compose([transforms.ToTensor()])
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

# Quantize the images
train_images = np.array([quantize_image(image.numpy().transpose(1, 2, 0) * 255) for image, _ in train_dataset])
train_labels = np.array(train_dataset.targets)

test_images = np.array([quantize_image(image.numpy().transpose(1, 2, 0) * 255) for image, _ in test_dataset])
test_labels = np.array(test_dataset.targets)

# Save the datasets
os.makedirs('./quantized_data', exist_ok=True)
save_dataset_as_npz(train_images, train_labels, './quantized_data/cifar10_train.npz')
save_dataset_as_npz(test_images, test_labels, './quantized_data/cifar10_test.npz')

print('Saved the dataset as .npz files')


class CIFAR10QuantizedDataset(Dataset):
    def __init__(self, npz_file):
        data = np.load(npz_file)
        self.images = data['data']
        self.labels = data['labels']

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx].astype(np.float32) / 255.0
        label = self.labels[idx]
        image = torch.tensor(image.transpose(2, 0, 1))  # HWC to CHW format
        label = torch.tensor(label, dtype=torch.long)
        return image, label

# Load the quantized dataset
train_dataset = CIFAR10QuantizedDataset('./quantized_data/cifar10_train.npz')
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

test_dataset = CIFAR10QuantizedDataset('./quantized_data/cifar10_test.npz')
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)


Files already downloaded and verified
Files already downloaded and verified
Saved the dataset as .npz files


In [32]:
import torch.optim as optim

def train(model, train_loader, optimizer, criterion):
    losses = []
    # ensure model is in training mode
    model.train()    
    
    for i, data in enumerate(train_loader, 0):        
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()   
                
        # forward pass
        output = model(images.float())
        # loss = criterion(output, labels.unsqueeze(1))
        loss = criterion(output, labels)
        
        # backward pass + run optimizer to update weights
        loss.backward()
        optimizer.step()
        
        # keep track of loss value
        losses.append(loss.data.cpu().numpy()) 
    return losses


def test(model, test_loader):    
    # ensure model is in eval mode
    model.eval() 
    y_true = []
    y_pred = []
   
    with torch.no_grad():
        for data in test_loader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            output = model(images.float())
            # run the output through sigmoid
            # output = torch.sigmoid(output_orig)  
            # compare against a threshold of 0.5 to generate 0/1
            # pred = (output.detach().cpu().numpy() > 0.5) * 1
            _, pred = torch.max(output.data, 1)
            labels = labels.cpu().float()
            y_true.extend(labels.tolist()) 
            y_pred.extend(pred.reshape(-1).tolist())
            # y_pred.extend((pred == labels).sum().item())
        
    return accuracy_score(y_true, y_pred)

num_epochs = 10
lr = 0.001 

def display_loss_plot(losses, title="Training loss", xlabel="Iterations", ylabel="Loss"):
    x_axis = [i for i in range(len(losses))]
    plt.plot(x_axis,losses)
    plt.title(title)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.show()

# loss criterion and optimizer
# criterion = nn.BCEWithLogitsLoss().to(device)
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=lr, betas=(0.9, 0.999))

# criterion = nn.CrossEntropyLoss()
# optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

# Setting seeds for reproducibility
torch.manual_seed(0)
np.random.seed(0)

running_loss = []
running_test_acc = []
t = trange(num_epochs, desc="Training loss", leave=True)

for epoch in t:
        loss_epoch = train(model, train_loader, optimizer,criterion)
        test_acc = test(model, test_loader)
        t.set_description("Training loss = %f test accuracy = %f" % (np.mean(loss_epoch), test_acc))
        t.refresh() # to show immediately the update           
        running_loss.append(loss_epoch)
        running_test_acc.append(test_acc)


Training loss = 1.681720 test accuracy = 0.391600: 100%|█████████████████| 10/10 [01:50<00:00, 11.06s/it]


In [33]:
test_accuracy = test(model, test_loader)
print('test accuracy = ', test_accuracy)

test accuracy =  0.3916


In [26]:
# # Save the Brevitas model to disk
# torch.save(model.state_dict(), "state_dict_LeNet_WeightActBias.pth")
# # torch.save(model, "LeNet_WeightActBias.pth")

In [29]:
# class QuantWeightActBiasLeNet(Module):
#     def __init__(self):
#         super(QuantWeightActBiasLeNet, self).__init__()
#         self.quant_inp = qnn.QuantIdentity(bit_width=4, return_quant_tensor=True)
#         self.conv1 = qnn.QuantConv2d(3, 6, 5, bias=True, weight_bit_width=weight_bit_width, bias_quant=Int32Bias)
#         self.relu1 = qnn.QuantReLU(bit_width=act_bit_width, return_quant_tensor=True)
#         self.conv2 = qnn.QuantConv2d(6, 16, 5, bias=True, weight_bit_width=weight_bit_width, bias_quant=Int32Bias)
#         self.relu2 = qnn.QuantReLU(bit_width=act_bit_width, return_quant_tensor=True)
#         self.fc1   = qnn.QuantLinear(16*5*5, 120, bias=True, weight_bit_width=weight_bit_width, bias_quant=Int32Bias)
#         self.relu3 = qnn.QuantReLU(bit_width=act_bit_width, return_quant_tensor=True)
#         self.fc2   = qnn.QuantLinear(120, 84, bias=True, weight_bit_width=weight_bit_width, bias_quant=Int32Bias)
#         self.relu4 = qnn.QuantReLU(bit_width=act_bit_width, return_quant_tensor=True)
#         self.fc3   = qnn.QuantLinear(84, 10, bias=True, weight_bit_width=weight_bit_width, bias_quant=Int32Bias)

#     def forward(self, x):
#         out = self.quant_inp(x)
#         out = self.relu1(self.conv1(out))
#         out = F.max_pool2d(out, 2)
#         out = self.relu2(self.conv2(out))
#         out = F.max_pool2d(out, 2)
#         out = out.reshape(out.shape[0], -1)
#         out = self.relu3(self.fc1(out))
#         out = self.relu4(self.fc2(out))
#         out = self.fc3(out)
#         return out

# model = QuantWeightActBiasLeNet()

####  Convert to ONNX model

In [7]:
# from brevitas.export import export_onnx_qcdq
# import torch

# # Weight-activation-bias model
# export_onnx_qcdq(quant_weight_act_bias_lenet, torch.randn(1, 3, 32, 32), export_path='./4b_weight_act_bias_lenet.onnx')
import torch
import onnx
from finn.util.test import get_test_model_trained
from brevitas.export import export_qonnx
from qonnx.util.cleanup import cleanup as qonnx_cleanup
from qonnx.core.modelwrapper import ModelWrapper
from finn.transformation.qonnx.convert_qonnx_to_finn import ConvertQONNXtoFINN
from qonnx.transformation.infer_shapes import InferShapes
from qonnx.transformation.fold_constants import FoldConstants
from qonnx.transformation.general import GiveReadableTensorNames, GiveUniqueNodeNames, RemoveStaticGraphInputs

# # cnv = get_test_model_trained("LeNet_WeightActBias.pth", 1, 1)
# model_path = "state_dict_LeNet_WeightActBias.pth"
# # model_path = "LeNet_WeightActBias.pth"
# state_dict = torch.load(model_path)  # This returns an OrderedDict
# model.load_state_dict(state_dict)  # Load the state dict into the model
# # model = torch.load(model_path)
# model.eval()

export_onnx_path = "LeNet_WeightActBias.onnx"
export_qonnx(model, torch.randn(1, 3, 32, 32), export_onnx_path,verbose=True)
qonnx_cleanup(export_onnx_path, out_file=export_onnx_path)

# torch.onnx.export(model, dummy_input, "model.onnx", verbose=True)

model = ModelWrapper(export_onnx_path)
model = model.transform(ConvertQONNXtoFINN())
model = model.transform(InferShapes())
model = model.transform(FoldConstants())
model = model.transform(GiveUniqueNodeNames())
model = model.transform(GiveReadableTensorNames())
model = model.transform(RemoveStaticGraphInputs())
model.save("LeNet_WeightActBias_tidy.onnx")

Exported graph: graph(%inp.1 : Float(1, 3, 32, 32, strides=[3072, 1024, 32, 1], requires_grad=0, device=cpu),
      %conv1.bias : Float(6, strides=[1], requires_grad=1, device=cpu),
      %conv2.bias : Float(16, strides=[1], requires_grad=1, device=cpu),
      %fc1.bias : Float(120, strides=[1], requires_grad=1, device=cpu),
      %fc2.bias : Float(84, strides=[1], requires_grad=1, device=cpu),
      %fc3.bias : Float(10, strides=[1], requires_grad=1, device=cpu)):
  %/quant_inp/act_quant/export_handler/Constant_output_0 : Float(requires_grad=0, device=cpu) = onnx::Constant[value={4}, onnx_name="/quant_inp/act_quant/export_handler/Constant"](), scope: __main__.QuantWeightActBiasLeNet::/brevitas.nn.quant_activation.QuantIdentity::quant_inp/brevitas.proxy.runtime_quant.ActQuantProxyFromInjector::act_quant/brevitas.export.onnx.qonnx.handler.BrevitasActQuantProxyHandler::export_handler # /home/inets/arish/FINN_arish/finn/deps/brevitas/src/brevitas/export/onnx/qonnx/handler.py:44:0
  %/quan

/home/inets/arish/FINN_arish/finn/deps/qonnx/src/qonnx/transformation/extract_conv_bias.py:52: UserWarning: Could not extract bias from node input: "Quant_0_out0"
input: "Quant_1_out0"
input: "Quant_2_out0"
output: "Conv_0_out0"
name: "Conv_0"
op_type: "Conv"
attribute {
  name: "dilations"
  ints: 1
  ints: 1
  type: INTS
}
attribute {
  name: "group"
  i: 1
  type: INT
}
attribute {
  name: "kernel_shape"
  ints: 5
  ints: 5
  type: INTS
}
attribute {
  name: "pads"
  ints: 0
  ints: 0
  ints: 0
  ints: 0
  type: INTS
}
attribute {
  name: "strides"
  ints: 1
  ints: 1
  type: INTS
}
doc_string: "/usr/local/lib/python3.10/dist-packages/torch/nn/modules/conv.py(458): _conv_forward\n/home/inets/arish/FINN_arish/finn/deps/brevitas/src/brevitas/nn/quant_conv.py(202): inner_forward_impl\n/home/inets/arish/FINN_arish/finn/deps/brevitas/src/brevitas/nn/quant_layer.py(318): forward_impl\n/home/inets/arish/FINN_arish/finn/deps/brevitas/src/brevitas/nn/quant_conv.py(198): forward\n/usr/local/l

In [8]:
from finn.util.visualization import showInNetron
showInNetron("LeNet_WeightActBias_tidy.onnx")

Serving 'LeNet_WeightActBias_tidy.onnx' at http://0.0.0.0:8081


In [9]:
from finn.util.pytorch import ToTensor
from qonnx.transformation.merge_onnx_models import MergeONNXModels
from qonnx.core.datatype import DataType

model = ModelWrapper("LeNet_WeightActBias_tidy.onnx")
global_inp_name = model.graph.input[0].name
ishape = model.get_tensor_shape(global_inp_name)
# preprocessing: torchvision's ToTensor divides uint8 inputs by 255
totensor_pyt = ToTensor()
chkpt_preproc_name = "LeNet_WeightActBias_tidy_preproc.onnx"
export_qonnx(totensor_pyt, torch.randn(ishape), chkpt_preproc_name)
qonnx_cleanup(chkpt_preproc_name, out_file=chkpt_preproc_name)
pre_model = ModelWrapper(chkpt_preproc_name)
pre_model = pre_model.transform(ConvertQONNXtoFINN())

# join preprocessing and core model
model = model.transform(MergeONNXModels(pre_model))
# add input quantization annotation: UINT8 for all BNN-PYNQ models
global_inp_name = model.graph.input[0].name
model.set_tensor_datatype(global_inp_name, DataType["UINT8"])

/home/inets/arish/FINN_arish/finn/deps/qonnx/src/qonnx/transformation/infer_data_layouts.py:127: UserWarning: Assuming 4D input is NCHW
  warnings.warn("Assuming 4D input is NCHW")


In [10]:
from qonnx.transformation.insert_topk import InsertTopK
from qonnx.transformation.infer_datatypes import InferDataTypes

# postprocessing: insert Top-1 node at the end
model = model.transform(InsertTopK(k=1))
chkpt_name = "LeNet_WeightActBias_tidy_prepost.onnx"
# tidy-up again
model = model.transform(InferShapes())
model = model.transform(FoldConstants())
model = model.transform(GiveUniqueNodeNames())
model = model.transform(GiveReadableTensorNames())
model = model.transform(InferDataTypes())
model = model.transform(RemoveStaticGraphInputs())
model.save(chkpt_name)

In [11]:
from finn.util.visualization import showInNetron
showInNetron("LeNet_WeightActBias_tidy_prepost.onnx")

Stopping http://0.0.0.0:8081
Serving 'LeNet_WeightActBias_tidy_prepost.onnx' at http://0.0.0.0:8081


### Streamline

In [29]:
from finn.transformation.streamline import Streamline
from qonnx.transformation.lower_convs_to_matmul import LowerConvsToMatMul
from qonnx.transformation.bipolar_to_xnor import ConvertBipolarMatMulToXnorPopcount
import finn.transformation.streamline.absorb as absorb
from finn.transformation.streamline.reorder import MakeMaxPoolNHWC, MoveScalarLinearPastInvariants
from qonnx.transformation.infer_data_layouts import InferDataLayouts
from qonnx.transformation.general import RemoveUnusedTensors
import finn.transformation.fpgadataflow.convert_to_hw_layers as to_hw
from qonnx.transformation.general import GiveReadableTensorNames, GiveUniqueNodeNames, RemoveStaticGraphInputs
from finn.transformation.streamline.round_thresholds import RoundAndClipThresholds


model = ModelWrapper("LeNet_WeightActBias_tidy_prepost.onnx")
model = model.transform(MoveScalarLinearPastInvariants())
model = model.transform(Streamline())
model = model.transform(LowerConvsToMatMul())
model = model.transform(MakeMaxPoolNHWC())
model = model.transform(absorb.AbsorbTransposeIntoMultiThreshold())
model = model.transform(ConvertBipolarMatMulToXnorPopcount())
model = model.transform(Streamline())
# model = model.transform(LowerConvsToMatMul())#
# model = model.transform(MakeMaxPoolNHWC())#
# model = model.transform(absorb.AbsorbTransposeIntoMultiThreshold())#
# model = model.transform(ConvertBipolarMatMulToXnorPopcount())#
# model = model.transform(Streamline())

# model = model.transform(absorb.AbsorbAddIntoMultiThreshold())
# model = model.transform(absorb.AbsorbMulIntoMultiThreshold())
# absorb final add-mul nodes into TopK
model = model.transform(absorb.AbsorbScalarMulAddIntoTopK())
# model = model.transform(RoundAndClipThresholds())
model = model.transform(GiveUniqueNodeNames())
# model = model.transform(GiveReadableTensorNames())
# model = model.transform(InferDataTypes())
model = model.transform(Streamline())

# bit of tidy-up
model = model.transform(InferDataLayouts())
model = model.transform(RemoveUnusedTensors())
model.save("LeNet_WeightActBias_tidy_streamlined.onnx")

In [30]:
from finn.util.visualization import showInNetron
showInNetron("LeNet_WeightActBias_tidy_streamlined.onnx")

Stopping http://0.0.0.0:8081
Serving 'LeNet_WeightActBias_tidy_streamlined.onnx' at http://0.0.0.0:8081


In [18]:
from finn.util.basic import pynq_part_map
# change this if you have a different PYNQ board, see list above
pynq_board = "Pynq-Z1"
fpga_part = pynq_part_map[pynq_board]
target_clk_ns = 10

import finn.transformation.fpgadataflow.convert_to_hw_layers as to_hw
from finn.transformation.fpgadataflow.create_dataflow_partition import (
    CreateDataflowPartition,
)
from finn.transformation.move_reshape import RemoveCNVtoFCFlatten
from finn.transformation.fpgadataflow.specialize_layers import SpecializeLayers
from qonnx.custom_op.registry import getCustomOp
from qonnx.transformation.infer_data_layouts import InferDataLayouts

model = ModelWrapper("LeNet_WeightActBias_tidy_streamlined.onnx")
model = model.transform(to_hw.InferBinaryMatrixVectorActivation())
model = model.transform(to_hw.InferQuantizedMatrixVectorActivation())
# TopK to LabelSelect
model = model.transform(to_hw.InferLabelSelectLayer())
# input quantization (if any) to standalone thresholding
model = model.transform(to_hw.InferThresholdingLayer())
model = model.transform(to_hw.InferConvInpGen())
model = model.transform(to_hw.InferStreamingMaxPool())
# get rid of Reshape(-1, 1) operation between hw nodes
model = model.transform(RemoveCNVtoFCFlatten())
# get rid of Tranpose -> Tranpose identity seq
model = model.transform(absorb.AbsorbConsecutiveTransposes())
# infer tensor data layouts
model = model.transform(InferDataLayouts())

parent_model = model.transform(CreateDataflowPartition())
parent_model.save("LeNet_WeightActBias_tidy_dataflow_parent.onnx")
sdp_node = parent_model.get_nodes_by_op_type("StreamingDataflowPartition")[0]
sdp_node = getCustomOp(sdp_node)
dataflow_model_filename = sdp_node.get_nodeattr("model")
# save the dataflow partition with a different name for easier access
# and specialize the layers to HLS variants
dataflow_model = ModelWrapper(dataflow_model_filename)
dataflow_model = dataflow_model.transform(SpecializeLayers(fpga_part))
dataflow_model.save("LeNet_WeightActBias_dataflow_model.onnx")

AssertionError: cycle-free graph violated: partition depends on itself